## Environment

In [1]:
%load_ext autoreload
%autoreload 2

In [8]:
# Basic
import os
import re
import sys
import glob
import numpy as np
import pandas as pd

import datetime
import itertools
import more_itertools
from pathlib import Path
from collections import defaultdict, Counter
from typing import (List, Dict, Any, NoReturn, 
                    Tuple, Optional, Union)
from tqdm import tqdm_notebook

import multiprocessing
from multiprocessing_logging import install_mp_handler

import logging
logging.basicConfig(level=logging.DEBUG,
                   format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

import warnings
warnings.filterwarnings('ignore')

print(f"Number CPU: {multiprocessing.cpu_count()}")

Number CPU: 12


In [9]:
# W2V
import spacy
import pymorphy2
from gensim.models import KeyedVectors
from gensim.models.phrases import Phrases
from nltk.stem.snowball import SnowballStemmer
from gensim.models.phrases import Phrases, npmi_scorer
from gensim.models import word2vec, keyedvectors

### Define paths

In [10]:
BASE_DIR = Path("..")
DATA_DIR  = BASE_DIR / "data" / 'ru_news'
LISTS_DIR  = BASE_DIR / "lists"
MODEL_DIR  = BASE_DIR / "models"

## Load data

In [11]:
df = []
for fn in tqdm_notebook(glob.glob(str(DATA_DIR / "*.csv"))):
    df_part = pd.read_csv(fn, sep=',', encoding='utf-8')
    print(f"Data part of shape: {df_part.shape}")
    df.append(df_part)
df = pd.concat(df).reset_index(drop=True)
print(f"All data of shape: {df.shape}")
df.head()

Data part of shape: (21673, 7)

All data of shape: (21673, 7)


,source,title,text,publication_date,rubric,subrubric,tags
0,lenta.ru,Синий богатырь,В 1930-е годы Советский Союз охватила лихорадк...,2020-08-30T00:01:00+03:00,Экономика,Госэкономика,NaN
1,lenta.ru,Загитова согласилась вести «Ледниковый период»,Олимпийская чемпионка по фигурному катанию Ал...,2020-08-31T20:04:00+03:00,Спорт,Зимние виды,NaN
2,lenta.ru,Объяснена опасность однообразного питания,Российский врач-диетолог Римма Мойсенко объясн...,2020-08-31T20:07:00+03:00,Из жизни,Еда,NaN
3,lenta.ru,«Предохраняться? А зачем?»,В 2019 году телеканал «Ю» запустил адаптацию з...,2020-08-30T00:04:00+03:00,Интернет и СМИ,ТВ и радио,NaN
4,lenta.ru,Ефремов систематически употреблял наркотики,Актер Михаил Ефремов систематически употребл...,2020-08-31T18:27:00+03:00,Культура,Кино,NaN


In [14]:
print(f"Unqiue sources: {df['source'].nunique()}")
df['source'].value_counts()

Unqiue sources: 4


ria.ru         14755
lenta.ru        4510
meduza.io       1538
tjournal.ru      870
Name: source, dtype: int64

**Rubrics and Subsubrics**

In [12]:
print(df.rubric.isna().value_counts())
print(f"\nUnique rubrics: {df.rubric.nunique()}, subrubrics: {df.subrubric.nunique()}")
print("\nRubrics:\n", df.rubric.value_counts())
print("\nSubrubrics:\n", df.subrubric.value_counts())

True     17170
False     4503
Name: rubric, dtype: int64

Unique rubrics: 15, subrubrics: 76

Rubrics:
 Россия               908
Мир                  718
Спорт                494
Экономика            420
Бывший СССР          394
Силовые структуры    220
Интернет и СМИ       211
Наука и техника      208
Культура             206
Из жизни             197
Путешествия          183
Ценности             141
Дом                  123
Нацпроекты            53
69-я параллель        27
Name: rubric, dtype: int64

Subrubrics:
 Общество        771
Политика        634
Госэкономика    206
Происшествия    202
Украина         199
               ... 
Движение          2
Вкусы             2
Реклама           1
Софт              1
Нацпроекты        1
Name: subrubric, Length: 76, dtype: int64


**Tags**

In [7]:
print(df.tags.isna().value_counts())
df.loc[df.tags.notnull()].head()

False    14752
True      6921
Name: tags, dtype: int64


,source,title,text,publication_date,rubric,subrubric,tags
6048,ria.ru,Попова оценила возможность второй волны COVID-19,"МОСКВА, 31 авг – РИА Новости. Глава Роспотре...",21:06 31.08.2020 (обновлено: 21:13 31.08.2020),NaN,NaN,"Распространение нового коронавируса, Коронавир..."
6049,ria.ru,"Ефремов заявил, что хочет ""попортить жизнь"" пр...","МОСКВА, 31 авг - РИА Новости. Актер Михаил Е...",18:46 31.08.2020 (обновлено: 18:47 31.08.2020),NaN,NaN,"ДТП с участием Михаила Ефремова, Михаил Ефремо..."
6050,ria.ru,Число случаев COVID-19 в США превысило шесть м...,"МОСКВА, 31 авг - РИА Новости. Число подтвержд...",18:50 31.08.2020,NaN,NaN,"Распространение нового коронавируса, Коронавир..."
6051,ria.ru,"Полуфиналы конкурса ""Лидеры России. Политика"" ...","МОСКВА, 31 авг - РИА Новости. Два очных полуф...",18:56 31.08.2020,NaN,NaN,"""Лидеры России"", Алексей Комиссаров, Общество,..."
6052,ria.ru,"Жириновский раскритиковал конфеты ""Ильич""","МОСКВА, 31 авг - РИА Новости. Лидер ЛДПР В...",19:03 31.08.2020 (обновлено: 19:08 31.08.2020),NaN,NaN,"Владимир Ленин (Владимир Ульянов), ЛДПР, Влади..."


**Date formats**

* `lenta.ru`: Datetime with timezone UTC+3H (Moscow), example: *2020-08-30T00:01:00+03:00*
* `meduza.io`: Datetime with human readable month name, example: *18:03, 31 августа 2020*
* `ria.ru`: Two datetimes. Fisrt - datetime of creation, second - datetime of update.
example: *21:06 31.08.2020  (обновлено: 21:13 31.08.2020)*
* `tjournal.ru `: Datetime as timestamp in seconds, example: *1598875929*

In [16]:
for source, group in df.groupby(by='source'):
    print(f"\nSource: {source} \nsample data format: {group.head()['publication_date'].to_list()}")


Source: lenta.ru 
sample data format: ['2020-08-30T00:01:00+03:00', '2020-08-31T20:04:00+03:00', '2020-08-31T20:07:00+03:00', '2020-08-30T00:04:00+03:00', '2020-08-31T18:27:00+03:00']

Source: meduza.io 
sample data format: ['18:03, 31 августа 2020', '10:22, 30 августа 2020', '10:43, 30 августа 2020', '09:28, 30 августа 2020', '11:09, 30 августа 2020']

Source: ria.ru 
sample data format: ['21:06 31.08.2020  (обновлено: 21:13 31.08.2020)', '18:46 31.08.2020  (обновлено: 18:47 31.08.2020)', '18:50 31.08.2020', '18:56 31.08.2020', '19:03 31.08.2020  (обновлено: 19:08 31.08.2020)']

Source: tjournal.ru 
sample data format: ['1598875929', '1598795150', '1598821788', '1598784941', '1598801069']


In [21]:
sample_tz = '2020-08-30T00:01:00+03:00'.split('+')[0]
datetime.datetime.strptime(sample_tz, '%Y-%m-%dT%H:%M:%S')

datetime.datetime(2020, 8, 30, 0, 1)

In [31]:
ru_months = {'января': '01', 
             'февраля': '02', 
             'марта': '03', 
             'апреля': '04', 
             'мая': '05', 
             'июня': '06',
             'июля': '07', 
             'августа': '08', 
             'сентября': '09', 
             'октября': '10', 
             'ноября': '11', 
             'декабря': '12'}

sample_ru = '18:03, 31 августа 2020'
m = re.search(r'[а-я]+', sample_ru).group(0)
datetime.datetime.strptime(re.sub(f" {m} ", f"-{ru_months.get(m)}-", sample_ru), '%H:%M, %d-%m-%Y')

datetime.datetime(2020, 8, 31, 18, 3)

In [ ]:
def convert_datetime(date):
    if type(date) == float:
        return None
    if len(date) <= 1:
        print(f"Error parcing date-time: {date}")
        return None

In [ ]:
def convert_datetime(date):
    if type(date) == float:
        return None
    if len(date) <= 1:
        print(f"Error parcing date-time: {date}")
        return None
    try:
        # Format: 2020-09-14 14:23:09
        dt = datetime.datetime.strptime(date, "%Y-%m-%d %H:%M:%S")
        return dt
    except:
        try:
            # Format: 2020-14-09 14:23:09
            dt = datetime.datetime.strptime(date, "%Y-%d-%m %H:%M:%S")
            return dt
        except:
            try:
                # Format: 14.09.2020 14:23
                dt = datetime.datetime.strptime(date, "%d.%m.%Y %H:%M")
                return dt
            except:
                try:
                # Format: 2020-09-21 06:28:29.000000
                    dt = datetime.datetime.strptime(date, "%Y-%m-%d %H:%M:%S.%f")
                    return dt
                except:
                    print(f"Error parcing date-time: {date} of type {type(date)}")
                    return None

## Configure preprocessing

In [23]:
def file_opener(filename: str) -> str:
    with open(filename, 'rt', encoding='utf-8-sig') as src:
        file = src.read()
    return "|".join([x for x in file.split('\n') if x])


class DataPreprocessorLemmatizer:

    text_features = ['msg']
    stopgrams = [
             'CONJ',   # союз
             'PRCL',   # частица
             'PRED',   # предикатив
             'NPRO',   # местоимение-сущ.
             'INTJ',   # междометие
             'Erro',   # ошибка
             'Dist',   # искажение
             'Ques',   # вопросительное слово
             'Dmns',   # указательное слово
             'Prnt'   # вводное слово
            ]

    def __init__(self, multipocess: bool, num_processors: int=16, chunksize: int=100, 
                stopwords_path: str = './lists'):
        # Language parsers
        self.__morph = pymorphy2.MorphAnalyzer()
        self.__nlp = spacy.blank('ru')
        # Multiprocessing params
        self.__multipocess = multipocess
        self.__num_processors = num_processors
        self.__chunksize = chunksize
        # Cleaning utils
        self.__nltk_stopwords = file_opener(os.path.join(stopwords_path, 'NLTK_stopwords.txt')).split("|")

    def get_stopwords(self):
        """ 
        Check intro-words list.
        """
        return self.__nltk_stopwords
    
    def get_analyzer(self):
        """
        Allow to access to Pymorphy Analyzer instance.
        """
        return self.__morph

    
    def _process_text(self, text: str):
        """ 
        Process single text and return list of tokens.
        """
        if pd.isna(text):
            return []
       # Pre-processing part 
        text = [str(token).lower()
                for token in self.__nlp.make_doc(text)
                if (token and token.is_alpha and len(str(token.text)) > 2 and ~token.is_stop)]
        # Processing part
        clean_text = []
        for token in text:
            token = self.__morph.parse(str(token).lower())[0]
            if ((token.normal_form not in self.__nltk_stopwords) 
                and all([tag not in token.tag for tag in self.stopgrams])):
                clean_text.append(token.normal_form)
        return clean_text
    

    def process_texts(self, texts: List[str]):
        """ 
        Process list of texts and return list of lists of tokens.
        """
        if self.__multipocess:
            with multiprocessing.Pool(self.__num_processors) as pool:
                processed_texts = list(tqdm_notebook(pool.imap(self._process_text, texts, 
                                                               chunksize=self.__chunksize), 
                                                     total=len(texts)))
            return processed_texts
        else:
            return [self._process_text(text) for text in tqdm_notebook(texts)]
        


    def process(self, data: pd.DataFrame,
                features_cols: Optional[List[str]] = None, copy: bool=True) -> pd.DataFrame:
        """
        Preprocess text for language modelling.
         - clean introduction words, numbers and small prefixes;
         - tokenize and lemmatize texts;
        """
        logging.info("Text processing started.")
        if not features_cols:
            features_cols = self.text_features

        for col_name in features_cols:
            logging.info(f"Processing '{col_name}' column...")
            data_processed = self.process_texts(data[col_name].fillna("").to_list())
            if not copy:
                data[col_name] = data_processed
            else:
                data[col_name + "_proc"] = data_processed

        logging.info("Text preprocessing finished.")
        return data

In [25]:
processor = DataPreprocessorLemmatizer(multipocess=False,
                                       num_processors=8, chunksize=200,
                                       stopwords_path=str(LISTS_DIR))

2021-07-21 17:33:27,590 - pymorphy2.opencorpora_dict.wrapper - INFO - Loading dictionaries from C:\Users\airen\Anaconda3\envs\pycharmenv\lib\site-packages\pymorphy2_dicts\data
2021-07-21 17:33:27,630 - pymorphy2.opencorpora_dict.wrapper - INFO - format: 2.4, revision: 393442, updated: 2015-01-17T16:03:56.586168


In [26]:
%%time
df = processor.process(data=df, features_cols=['text'])

2021-07-21 17:33:42,031 - root - INFO - Text processing started.
2021-07-21 17:33:42,032 - root - INFO - Processing 'text' column...


2021-07-21 17:46:12,296 - root - INFO - Text preprocessing finished.



Wall time: 12min 30s


In [27]:
for i, row in df[['text', 'text_proc']].head().iterrows():
    print("--------------")
    print(f"{row['text']}\n{row['text_proc']}")

--------------
В 1930-е годы Советский Союз охватила лихорадка — в десятилетие бурной индустриализации повсюду гремели сообщения о новых трудовых подвигах простого народа ради построения коммунизма и светлого будущего. Первым из них стал шахтер из Донбасса  Алексей Стаханов . 85 лет назад он установил рекорд по добыче угля за смену и на следующее утро проснулся знаменитым на весь мир. Его одаривали квартирами и машинами, возили по Союзу, он стал любимцем Сталина, однако в итоге не выдержал славы и спился. Фамилия Стаханова, которая дала название движению трудовиков, превратилась в нарицательную и продолжила существовать отдельно от него. Живая витрина коммунизма — в материале  «Ленты.ру» . История легенды советского труда началась на шахте «Центральная-Ирмино» в Донбассе. К 1935 году советские власти запустили вторую пятилетку и активно проводили политику ускоренной индустриализации, пытаясь по заветам Владимира Ленина догнать и перегнать развитые капиталистические страны, от которых б

In [28]:
class Bigrammer:
    
    def __init__(self, phrase_model=None):
        self.__phrase_model = phrase_model
    
    def train(self, texts: List[List[str]], 
              min_count: int, threshold: float,
             to_save: bool, save_path: str=".", phrases_fn: str="phrases.pkl"):
        """
        Train gensim Phrases model with NPMI scorer.
        :param texts - The training corpus must be a sequence of sentences,
                        with each sentence a list of tokens.
        :param min_count – Ignore all words and bigrams with total 
                            collected count lower than this value.
        :param threshold – Represent a score threshold for forming 
                            the phrases (higher means fewer phrases). 
                            A phrase of words a followed by b is accepted if the score of 
                            the phrase is greater than threshold. 
                            For NPMI scorer is in the range -1 to 1.
        """
        logging.info("Training bigrammer started.")
        self.__phrase_model = Phrases(texts, min_count=min_count, 
                               threshold=threshold, scoring='npmi')
        logging.info("Training bigrammer finished.")
        if to_save:
            self.__phrase_model.save(os.path.join(save_path, phrases_fn))
            if os.path.isfile(os.path.join(save_path, phrases_fn)):
                logging.info(f"Bigrammer model successfully saved to: {os.path.join(save_path, phrases_fn)}")
        return self
    
    @classmethod
    def load(cls, save_path: str, phrases_fn: str) -> object:
        """
        Load pre-trained model from file and init.
        """
        if os.path.isfile(os.path.join(save_path, phrases_fn)):
            logging.info(f"Bigrammer model loading from: {os.path.join(save_path, phrases_fn)}")
        phrase_model = Phrases.load(os.path.join(save_path, phrases_fn))
        logging.info(f"Bigrammer model successfully loaded.")
        return cls(phrase_model=phrase_model)
    
    
    def create_bigramms(self, texts: List[List[str]]) -> List[List[str]]:
        """
        Create bi-gramms from given text data, already splitted.
        """
        return [self.__phrase_model[text] if len(text) > 0 
                else [] for text in tqdm_notebook(texts)]
    
    
    def process(self, data: pd.DataFrame,
                text_col: str, copy: bool=True) -> pd.DataFrame:
        """
        Create bi-gramms from given column in dataframe.
        """
        logging.info(f"Bigramms creation for texts in column {text_col} started")
        data_processed = self.create_bigramms(data[text_col].fillna("").to_list())
        if not copy:
            data[text_col] = data_processed
        else:
            data[text_col + "_bigramms"] = data_processed

        logging.info("Bigramms creation finished.")
        return data
    
    
    def get_vocab(self) -> Dict[bytes, int]:
        logging.info(f"Bigrammer vocab size: {len(self.__phrase_model.vocab)}")
        return self.__phrase_model.vocab
    
    
    def get_phraser(self) -> Phrases:
        return self.__phrase_model

In [30]:
bigrammer = Bigrammer().train(df['text_proc'].to_list(),
                             min_count=5, threshold=0.3,
                             to_save=False)
bi_vocab = bigrammer.get_vocab()

2021-07-21 17:46:41,567 - root - INFO - Training bigrammer started.
2021-07-21 17:46:41,568 - gensim.models.phrases - INFO - collecting all words and their counts
2021-07-21 17:46:41,568 - gensim.models.phrases - INFO - PROGRESS: at sentence #0, processed 0 words and 0 word types
2021-07-21 17:46:43,944 - gensim.models.phrases - INFO - PROGRESS: at sentence #10000, processed 1594614 words and 996705 word types
2021-07-21 17:46:46,435 - gensim.models.phrases - INFO - PROGRESS: at sentence #20000, processed 3207485 words and 1626771 word types
2021-07-21 17:46:46,863 - gensim.models.phrases - INFO - collected 1729018 word types from a corpus of 3467579 words (unigram + bigrams) and 21673 sentences
2021-07-21 17:46:46,864 - gensim.models.phrases - INFO - using 1729018 counts as vocab in Phrases<0 vocab, min_count=5, threshold=0.3, max_vocab_size=40000000>
2021-07-21 17:46:46,864 - root - INFO - Training bigrammer finished.
2021-07-21 17:46:46,865 - root - INFO - Bigrammer vocab size: 1729

In [31]:
bigramms = [k for k in bi_vocab.keys() if "_" in k.decode('utf-8')]
print(f"Bigramms in vocab: {len(bigramms)} from {len(bi_vocab)} which is {100*len(bigramms)/len(bi_vocab)}\n")

for i, (k, v) in enumerate(bi_vocab.items()):
    print(f"{k.decode('utf-8')} --> {v}")
    if i > 10:
        break

Bigramms in vocab: 1645614 from 1729018 which is 95.17622141585571

год --> 35636
советский --> 1935
год_советский --> 53
союз --> 1388
советский_союз --> 416
охватить --> 186
союз_охватить --> 1
лихорадка --> 55
охватить_лихорадка --> 1
десятилетие --> 236
лихорадка_десятилетие --> 1
бурный --> 44


In [32]:
%%time
df = bigrammer.process(df, text_col='text_proc', copy=True)

2021-07-21 17:47:00,790 - root - INFO - Bigramms creation for texts in column text_proc started


2021-07-21 17:47:12,546 - root - INFO - Bigramms creation finished.



Wall time: 11.8 s


In [33]:
for i, row in df[['text', 'text_proc_bigramms']].head().iterrows():
    print("--------------")
    print(f"{row['text']}\n{row['text_proc_bigramms']}")

--------------
В 1930-е годы Советский Союз охватила лихорадка — в десятилетие бурной индустриализации повсюду гремели сообщения о новых трудовых подвигах простого народа ради построения коммунизма и светлого будущего. Первым из них стал шахтер из Донбасса  Алексей Стаханов . 85 лет назад он установил рекорд по добыче угля за смену и на следующее утро проснулся знаменитым на весь мир. Его одаривали квартирами и машинами, возили по Союзу, он стал любимцем Сталина, однако в итоге не выдержал славы и спился. Фамилия Стаханова, которая дала название движению трудовиков, превратилась в нарицательную и продолжила существовать отдельно от него. Живая витрина коммунизма — в материале  «Ленты.ру» . История легенды советского труда началась на шахте «Центральная-Ирмино» в Донбассе. К 1935 году советские власти запустили вторую пятилетку и активно проводили политику ускоренной индустриализации, пытаясь по заветам Владимира Ленина догнать и перегнать развитые капиталистические страны, от которых б

## TextRank

In [34]:
import networkx as nx

In [ ]:
class Candidate(object):
    """
    The keyphrase candidate data structure.
    """

    def __init__(self, token: str, 
                 is_bigramm: bool,
                 pos: pymorphy2.tagset.OpencorporaTag,
                 score: float,
                 lexical_forms: Dict[str, str]=None,
                 bigramm_sep: str="_"):

        self._token = token
        self._is_bigramm = is_bigramm
        self._pos = pos
        self._score = score
        
        self._lexical_forms = lexical_forms 
        self._bigramm_splitted = [] # TODO: split bigramm
        
        if self._is_bigramm:
            self._bigramm_splitted = self._token.split(bigramm_sep)
            
    def __str__(self):
        return f"Token: {self._token} with score: {self._score}, is bigramm: {self._is_bigramm} with POS: {self._pos}"

In [36]:
class TextRanker:
    """
    TextRank for keyword extraction.
    This model builds a graph that represents the text. A graph based ranking
    algorithm is then applied to extract the lexical units (here the words) that
    are most important in the text.
    In this implementation, 
     - nodes - are words of certain part-of-speech (nouns/adjectives/..) 
     - edges - represent co-occurrence relation, controlled by the distance 
               between word occurrences - a window of N words). 
    """
    rus_lexical_forms = ['nomn', 'gent', 'datv', 'accs', 'ablt', 'loct', 'voct']

    def __init__(self):
        # From pymorphy2 avaliable POS tags
        # ref: http://opencorpora.org/dict.php?act=gram 
        self.__pos_list = ['NOUN', 'ADJS', 'ADJF', 'COMP', 'VERB', 'INFN', 
                           'PRTF', 'PRTS', 'GRND', 'NUMR', 'ADVB', 'Abbr']
        
        # Russian language parser
        self.__morph = pymorphy2.MorphAnalyzer()
        # Words graph
        self.__graph = nx.Graph()
        self.__texts = []  # as List[List[Dict[str, Any]]] -> [[{'words': [], 'bigramm': [], 'POS': []}]]
        # Each inner Dict == single token
        # Each inner List == single text
        # Outer List is composition of texts
        
        # Keyphrase candidates container
        self.__candidates = defaultdict(Candidate)
        

        
    def candidate_weighting(self, texts: List[List[str]],
                            window: int=2, pos_list: List[str]=None,
                            include_bigramms: bool=True,
                            top_percent: float=None):
        """
        Tailored candidate ranking method for TextRank. 
        Keyphrase candidates are either composed from the T-percent (top_percent) 
        highest-ranked words or extracted using the `candidate_selection()` method.
        Candidates are ranked using the sum of their words.
        :param window - the window for connecting words in the graph.
        :param pos_list - the set of valid pos for words to be considered as nodes
                    in the graph, defaults to ('NOUN', 'PROPN', 'ADJ').
        :param top_percent - percentage of top vertices to keep for phrase generation.
        """
        if pos_list is not None:
            self.__pos_list = pos_list
            
        # flatten document as a sequence of (word, bigramm, pos) samples
        self.__texts = self.__tag_words(texts)
        self.__window = window
        self.__include_bigramms = include_bigramms
        self.__build_word_graph()
        
        # Computes the word scores using the unweighted PageRank formula
        # pagerank_scipy() is a SciPy sparse-matrix implementation of the power-method
        # Returns: pagerank – Dictionary of nodes with PageRank as value
        textranked = nx.pagerank_scipy(self.__graph, alpha=0.85, tol=0.0001, weight=None)
        
        # Generate the phrases from the T-percent top ranked words
        if top_percent is not None:

            # warn user as this is not the pke way of doing it
            logging.info(f"Candidates are generated using {top_percent}%-top")

            # computing the number of top keywords
            n_nodes = self.__graph.number_of_nodes()
            to_keep = min(int(n_nodes * top_percent), n_nodes)

            # Sorting the nodes by decreasing scores
            top_words = {k: v for k, v in sorted(textranked.items(), key=lambda item: item[1], reverse=True)}
            
            # Create candidates
            self.__create_candidates(top_words)
    
    
    def __check_validness(self, token_dict: Dict[str, Any]) -> bool:
        """
        Mark token `valid` if it belongs to one of selected POS or it is a bigramm.
        """
        return token_dict['bigramm'] or (token_dict['pos'] is not None and any([tag in token_dict['pos'] 
                                         for tag in self.__pos_list]))
        
        
    def __tag_words(self, texts: List[List[str]]):
        """
        Process given texts to selected form: 
        [[{'words': [], 'bigramm': [], 'POS': [], 'valid'}]]
        """
        texts = [[{'token': token,
                  'bigramm': True if "_" in token else False,
                  'pos': self.__morph.parse(str(token).lower())[0].tag if "_" in token else None}
                 for token in tokens] 
                for tokens in tqdm_notebook(texts)]
        _ = [[token.update({'valid': self.__check_validness(token)})  
                 for token in tokens] for tokens in tqdm_notebook(texts)]
        return texts
            
            
    def __build_word_graph(self):
        """
        Build a graph representation of the document in which nodes/vertices
        are words and edges represent co-occurrence relation. Syntactic filters
        can be applied to select only words of certain Part-of-Speech.
        Co-occurrence relations can be controlled using the distance between
        word occurrences in the document.
        """
        tokens = itertools.chain.from_iterable(self.__texts)
        # add nodes to the graph
        logging.info(f"Adding nodes to graph...")
        self.__graph.add_nodes_from([token['token'] for token in tokens if token['valid']])

        # add edges to the graph
        logging.info(f"Adding edges...")
        for text_i, tokens in enumerate(self.__texts):
            for token_i, token in enumerate(tokens):
                # speed up things
                if not token['valid']:
                    continue
                start_ind = min(token_i, (self.__window - 1) // 2)
                end_ind = min(i + self.__window, len(tokens))
                for j in range(start_ind, end_ind):
                    linked_token = tokens[j]
                    if linked_token['valid'] and linked_token['token'] != token['token']:
                        self.__graph.add_edge(token['token'], linked_token['token'])
        logging.info(f"Adding edges ended.")
        
        
    def __create_candidates(self, scored_tokens: Dict[str, float]):
        """
        Create keywords dict for all texts.
        """        
        for token, score in tqdm_notebook(scored_tokens.items()):
            self.__add_candidate(token, score)
        logging.info(f"Keywords candidates created: {len(self.__candidates)}")
        
        
    def __add_candidate(self, token: str, score: float=None):
        """
        Add a keyphrase candidate to the candidates container.
        """
        is_bigramm = True if "_" in token else False
        lexical_forms = defaultdict(str)
        parsed_token = self.__morph.parse(str(token).lower())[0]
        pos = parsed_token.tag
        if ~is_bigramm and (('NOUN' in pos) or ('ADJF' in pos) 
                            or ('NUMR' in pos) or ('PRTF' in pos)):
            for form in self.rus_lexical_forms:
                lexical_forms[form] = parsed_token.inflect({form}).word
        pos = parsed_token.tag
        self.__candidates[token] = Candidate(token, is_bigramm, pos, lexical_forms, score)
        
        
    def get_candidates(self) -> Dict[str, Any]:
        return self.__candidates
    
    def select_keywords_from_text(self, text: List[str], return_scores: bool=False) -> List[str]:
        return [candidate for candidate in self.__candidates if candidate in text]
            
    def select_keywords_from_texts(self, texts: List[List[str]], return_scores: bool=False) -> List[List[str]]:
        return [self.select_keywords_from_text(text) for text in texts]

In [37]:
textrank = TextRanker()
textrank.candidate_weighting(texts=df['text_proc_bigramms'].to_list(),
                            window=5, include_bigramms=True,
                            top_percent=0.2)

2021-07-21 17:47:40,601 - pymorphy2.opencorpora_dict.wrapper - INFO - Loading dictionaries from C:\Users\airen\Anaconda3\envs\pycharmenv\lib\site-packages\pymorphy2_dicts\data
2021-07-21 17:47:40,645 - pymorphy2.opencorpora_dict.wrapper - INFO - format: 2.4, revision: 393442, updated: 2015-01-17T16:03:56.586168


2021-07-21 17:50:29,671 - root - INFO - Adding nodes to graph...


2021-07-21 17:50:30,102 - root - INFO - Adding edges...
2021-07-21 17:50:32,512 - root - INFO - Adding edges ended.
2021-07-21 17:50:35,819 - root - INFO - Candidates are generated using 0.2%-top


2021-07-21 17:50:57,394 - root - INFO - Keywords candidates created: 33263


In [38]:
keywords_candidates = textrank.get_candidates()
keywords_candidates

defaultdict(__main__.Candidate,
            {'риа_новость': <__main__.Candidate at 0x21c456b3668>,
             'комментарий_закладка': <__main__.Candidate at 0x21c456b3710>,
             'дональд_трамп': <__main__.Candidate at 0x21c456b35f8>,
             'президент_сша': <__main__.Candidate at 0x21c456b3240>,
             'владимир_путин': <__main__.Candidate at 0x21c456b3630>,
             'премьер_министр': <__main__.Candidate at 0x21c46871208>,
             'президент_россия': <__main__.Candidate at 0x21c468711d0>,
             'коронавирусный_инфекция': <__main__.Candidate at 0x21c46871278>,
             'пандемия_коронавирус': <__main__.Candidate at 0x21c46871470>,
             'сообщить_риа': <__main__.Candidate at 0x21c46871d68>,
             'год_назад': <__main__.Candidate at 0x21c468715c0>,
             'случай_заражение': <__main__.Candidate at 0x21c46871400>,
             'июнь_год': <__main__.Candidate at 0x21c46871be0>,
             'февраль_год': <__main__.Candidate at

In [39]:
df['keywords'] = df['text_proc_bigramms'].apply(textrank.select_keywords_from_text)
df.head()

KeyboardInterrupt: 

In [ ]:
for i, row in df[['text', 'keywords']].head().iterrows():
    print("--------------")
    print(f"{row['text']}\n{row['keywords']}")